In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

# Creating a ANN using Pytorch

In [ ]:
df=pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:

df['Outcome']=np.where(df['Outcome']==1,'Diabetic','No Diabetic')

In [ ]:
sns.pairplot(df,hue='Outcome')

In [ ]:

X=df.drop('Outcome',axis=1).values
y=df['Outcome'].values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
# Creating some Tensors
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

In [ ]:
# Creatng the model in pytorch
class ANN_Model(nn.Module):
    def __init__(self,input_features=8,hidden1=20,hidden2=20,output_features=2):
        super().__init__()
        self.fc1=nn.Linear(input_features,hidden1)
        self.fc2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,output_features)
        
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.out(x)
        return x
    
        

In [ ]:
#Instantiate my ANN
torch.manual_seed(4206969)
model=ANN_Model()

In [ ]:
model.parameters

In [ ]:
# Backward Propogation -- define the loss func and the optimizer 
loss_function=nn.CrossEntropyLoss() #for multi class 
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)


In [ ]:
# trigger the forward propogation
epochs=500
final_losses=[]
for i in range(epochs):
    i+i+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if(i%10==1):
        print('Epochs number :{}and the loss is {}'.format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
#plotting the loss funtion
plt.plot(range(epochs),final_losses)
plt.ylabel('loss')
plt.xlabel('Epochs')


In [ ]:
#predictions
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())
        

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual')
plt.ylabel('Predicted')

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)

In [ ]:
print(score)

In [ ]:
# save the model
torch.save(model,'diabetes.pt')

In [ ]:
model=torch.load('./diabetes.pt')

In [ ]:
model.eval()